# Snorkel Workshop: Extracting Spouse Relations from the News

## Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# Connect to the database backend and initalize a Snorkel session
from lib.init import *

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, load_as_array=True, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

In [ ]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=17255  #epochs=10  batch size=256
[reRNN] Epoch 0 (44.80s)	Average loss=0.582855	Dev F1=3.67
[reRNN] Epoch 1 (94.16s)	Average loss=0.543309	Dev F1=32.64
[reRNN] Epoch 2 (154.77s)	Average loss=0.536457	Dev F1=37.20
[reRNN] Epoch 3 (219.99s)	Average loss=0.535602	Dev F1=36.36
[reRNN] Epoch 4 (288.14s)	Average loss=0.535083	Dev F1=35.78
[reRNN] Epoch 5 (368.77s)	Average loss=0.534794	Dev F1=36.89
[reRNN] Epoch 6 (431.79s)	Average loss=0.534677	Dev F1=36.84
[reRNN] Epoch 7 (499.04s)	Average loss=0.534586	Dev F1=36.32
[reRNN] Model saved as <reRNN>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [ ]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [ ]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!